# EXTRACCIÓN DE VUELOS

## En este Notebook se va a desarrollar el proceso de extracción de todos los vuelos salientes del *Aeropuerto Adolfo Suárez Madrid-Barajas* en el periodo comprendido entre el 01-11-2022 y el 31-10-2023, es decir, 1 año completo.

## Para ello se va a utilizar la técnica de extracción de datos "webscrapping" sobre la web [flightera](https://www.flightera.net/en/airport/Madrid/LEMD/departure) que mantiene registros de vuelos desde al menos el año 2017.

### Las librerías que se van a emplear principalmente para el proceso son:
- **selenium** 
- **pandas** 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
import pandas as pd

In [2]:
import time
import numpy as np
import pylab as plt   
import seaborn as sns


## Para evitar problemas en el proceso de extracción se van a activar las opciones de **adblock** (previa descarga del driver correspondiente) y de las **cockies**.

In [3]:
# Options
opciones=Options()

opciones.add_extension('drivers/adblock.crx')       # adblocker
opciones.add_argument('cookies=cookies')    # man



## Colocamos el driver en la prímera página que queremos *scrappear*, la correspondiente a las últimas horas de 31-10-2023

In [4]:
driver = webdriver.Chrome(options = opciones)
url = 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2023-10-31%2023_00?'
driver.get(url)


## Clickamos sobre el botón de aceptar las coockies que aparece al cargar la página

In [6]:
#aceptar = driver.
aceptar = driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
aceptar.click()


## Empezamos a almacenar los datos correspondientes de la primera página sobre una tabla que llamamos *flights*.

### Esta web presenta unos 35 registros de vuelos por cada página.

In [132]:
flights = [flight.text.split('\n') for flight in driver.find_elements(By.XPATH, '//table//tbody//tr')[1:]]

## Clickamos sobre el boton de *Earlier Flights* que aparece en la página para cargar los siguientes 35 vuelos.

In [7]:
aceptar = driver.find_element(By.XPATH,'/html/body/div[3]/main/div[6]/div[1]')
aceptar.click()

## Aparece un nuevo *pop up* para hacer donaciones a la web, por lo que clickamos sobre el boton *"Continue without supporting us"*

In [8]:
aceptar2 = driver.find_element(By.XPATH,'/html/body/div[8]/div/div/div/div/div/div[3]/span[1]/button')
aceptar2.click()

# Una vez estudiada la dinámica de la web, se hace un cálculo del número de nuevas páginas que va a haber que abrir en base al promedio de salidas diarias del aeropuerto de Barajas (en este se opta por 7000, que cubrirá los registros necesarios con algo de holgura).

## Además, puesto que el *scrappeo* se hace visual, se podrá interrumpir el bucle una vez se observe que hemos pasado el día 01-11-2022

## El bucle copiará los registros de cada página en la tabla *flights* y a continuación hara *click* en el botón de "Earlier Flights"

## Se ha observado tambien, que el pop up de donaciones aparece periódicamente, por lo que se introduce in bloque *try -except* en el bucle para que en caso de aparecer, clicke de nuevo sobre el botón



In [160]:

for i in range(7000):
    try:
        flights += [flight.text.split('\n') for flight in driver.find_elements(By.XPATH, '//table//tbody//tr')[1:]]

        aceptar = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[6]/div[1]/a')
        aceptar.click()
        
    except:
        aceptar2 = driver.find_element(By.XPATH,'/html/body/div[8]/div/div/div/div/div/div[3]/span[1]/button')
        aceptar2.click()
        
    





NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[8]/div/div/div/div/div/div[3]/span[1]/button"}
  (Session info: chrome=119.0.6045.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF77FAA82B2+55298]
	(No symbol) [0x00007FF77FA15E02]
	(No symbol) [0x00007FF77F8D05AB]
	(No symbol) [0x00007FF77F91175C]
	(No symbol) [0x00007FF77F9118DC]
	(No symbol) [0x00007FF77F94CBC7]
	(No symbol) [0x00007FF77F9320EF]
	(No symbol) [0x00007FF77F94AAA4]
	(No symbol) [0x00007FF77F931E83]
	(No symbol) [0x00007FF77F90670A]
	(No symbol) [0x00007FF77F907964]
	GetHandleVerifier [0x00007FF77FE20AAB+3694587]
	GetHandleVerifier [0x00007FF77FE7728E+4048862]
	GetHandleVerifier [0x00007FF77FE6F173+4015811]
	GetHandleVerifier [0x00007FF77FB447D6+695590]
	(No symbol) [0x00007FF77FA20CE8]
	(No symbol) [0x00007FF

## Una vez finalizado el *scrappeo* se comprueba el númos de filas recogidas, en este caso *223405 filas*.

In [172]:
len(flights)

223405

# Se estudia el número de elementos(columnas) recogidas en cada registro y se genera un DataFrame con los datos en crudo, y nombres de columnas provisionales, para tener una base para la posterior limpieza.

In [163]:
pd.set_option('display.max_rows', None)
columns =["bad_date","1", "status","cod_flight","airliner", "destiny", "depart","delay_d","8","arrival","delay_a","duration","12"]

In [165]:
flights = pd.DataFrame(flights, columns=columns)
flights_ori = flights

## Guardamos los datos en crudo en un archivo *.csv*

In [180]:
flights.to_csv("../data/flights/raw_flights.csv", index=False)